# Pipeline to select pathogenic SNPs in a CAS12 seed region

In [1]:
import gzip
import re
import sys
import pysam

First load the snps of interest from the clinvar database. Make sure clinvar.vcf.gz is available in the same directory as the notebook.

In [ ]:
print("\nReading ClinVar database for fun")

snps=[]
with gzip.open("clinvar.vcf.gz","rt") as vcf_file:        
        print("selecting all pathogenic SNVs")
        for line in vcf_file:
            # Check if the line starts with a "#" character, which indicates a comment line
            if line[0] == "#":
                continue

            # Split the line into fields
            fields = line.strip().split("\t")
            
            # Split the info field into substrings
            info_substrings = fields[7].split(";")
     
            # Initialize flags to track whether the CLNVC and SLNSIG conditions are met
            clnvc_found = False
            clnsig_found = False
         
            # Initialize a variable to store the gene name & frequency
            gene = ""
            frequency = ""
            
           # Loop through the info substrings
            for substring in info_substrings:
                # Check if the CLNVC condition is met
                if substring == "CLNVC=single_nucleotide_variant":
                    clnvc_found = True
                # Check if the SLNSIG condition is met
                if "CLNSIG=Pathogenic" in substring:
                    clnsig_found = True
                    # Check if the GENEINFO condition is met
                elif substring.startswith("GENEINFO="):
                    # Extract the gene name from the substring
                    gene = substring.split("=")[1]
            # Check if the CLNALLE condition is met
                elif substring.startswith("AF_EXAC="):
                    # Extract the clinical frequency from the substring
                    frequency = substring.split("=")[1]
            
            # If both conditions are met, write the candidate to the CSV file
            if clnvc_found and clnsig_found:
                snps.append(([fields[0], gene, fields[1], fields[3], fields[4], frequency]))
                #writer.writerow([fields[0], gene, fields[1], fields[3], fields[4], frequency])


Reading ClinVar database for fun
selecting all pathogenic SNVs


Now for every SNP check if there's a PAM site nearby.

First load the entire human genome into memory for quick lookup. Make sure GRCh38.fa.gz is located in the same directory as the notebook.

In [ ]:
hg={}
with gzip.open("GRCh38.fa.gz", "rt") as genome_file:
    seq,chrom="",None
    for line in genome_file:
        if line[0]=='>':
            if seq!="" and chrom!=None:
                hg[chrom]=seq
            p=re.compile("(.*) Homo sapiens chromosome (\d{1,2}|X|Y), GRCh38.p14 Primary Assembly")
            match=p.search(line)
            if match!=None: #actual primary assembly
                ctgname=match.group(1)
                chrom=match.group(2)
                print(chrom)
            else:
                chrom=None  
            seq=""
        else:
            seq+=line[:-1]
hg['MT']=seq

1


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
X
Y


In [15]:
p=re.compile("(TTT[A,C,G])|([C,G,T]AAA)")
with gzip.open('pamsites.bed.gz','wt') as bed:
    for chrom in hg.keys():
        for m in p.finditer(hg[chrom]):
            if chrom=='MT':
                chrom='M'
            o='-' if m.groups()[0]==None else '+'
            bed.write("chr%s\t%d\t%d\t%s\n"%(chrom, m.start(), m.end(), o))


# Intersect with SNPs in 1000 genomes data, to obtain non-overlapping PAM sites

Download 1000 genomes vcf with variants and allele frequencies, e.g. through:

ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/release/20181203_biallelic_SNV/ALL.wgs.shapeit2_integrated_v1a.GRCh38.20181129.sites.vcf.gz

And filter by allele frequency:

In [ ]:
!bcftools view -i 'INFO/AF>=0.01 & INFO/AF<=0.99' -Oz -o ALL.wgs.shapeit2_integrated_v1a.GRCh38.20181129.sites.AF01.vcf.gz ALL.wgs.shapeit2_integrated_v1a.GRCh38.20181129.sites.vcf.gz

Make sure bedtools is available from your notebook

In [6]:
!bedtools intersect -v -wa -a pamsites.bed.gz -b ALL.wgs.shapeit2_integrated_v1a.GRCh38.20181129.sites.AF01.snps.vcf.gz | gzip > pamsites.nooverlap.bed.gz

/bin/bash: bedtools: command not found


Write all seed sites to a bed file.

In [13]:
seedsize=5

with gzip.open('seedsites.%d.bed.gz'%seedsize,'wt') as bedout:
    with gzip.open('pamsites.nooverlap.bed.gz','rt') as bed:
        for line in bed:
            chrom,start,end,o=line[:-1].split('\t')
            if o=='+':
                bedout.write("%s\t%s\t%d\t%s\n"%(chrom, end, int(end)+seedsize, o))
            else:
                bedout.write("%s\t%d\t%s\t%s\n"%(chrom, int(start)-seedsize, start, o))

In [ ]:
!bedtools sort -i seedsites.5.bed.gz | merge -i - | gzip > seedsites.merged.bed.gz

# Intersect clinvar with merged seedsites to obtain CAS12 targetable SNPs

In [ ]:
!bcftools annotate --rename-chrs chr.map clinvar_20210731.vcf.gz | bcftools view -i 'INFO/CLNSIG="Pathogenic"' -v snps | bedtools intersect -header -a - -b seedsites.merged.bed.gz | bcftools view - --write-index -Oz -o clinvar_20210731.pathogenic.snps.CAS12targetable.vcf.gz

clinvar_20210731.pathogenic.snps.CAS12targetable.vcf.gz now contains all pathogenic SNPs in clinvar that can be targeted by CAS12.

In [19]:
n/len(snps)

0.0

# Annotate vcf with target sequence and relative location to PAM site

In [14]:
revcomptable = str.maketrans("acgtACGTRY","tgcaTGCAYR")
def revcomp(s):
    return s.translate(revcomptable)[::-1]


Read every record in the filtered clinvar vcf and write to an annotated vcf.

In [17]:
vcffile="clinvar_20210731.pathogenic.snps.CAS12targetable.vcf.gz"

In [20]:
posPAM =  ["TTTC", "TTTA", "TTTG"]
negPAM  = ["CAAA", "TAAA", "GAAA"]

seedsize=5
pamsize=4

w=seedsize+pamsize

n=0

with pysam.VariantFile(vcffile) as vcfreader:
    vcfreader.header.info.add('SNP_position', 1, 'String', 'Position of SNP relative to PAM site')
    vcfreader.header.info.add('CRISPR_ref_target_sequence', 1, 'String', 'CRISPR target sequence to address the reference (WT) allele.')
    vcfreader.header.info.add('CRISPR_alt_target_sequence', 1, 'String', 'CRISPR target sequence to address the alternative (pathogenic) allele.')
    vcfreader.header.info.add('STRAND', 1, 'String', 'Whether PAM site is found on the positive (+) or negative (-) strand.')
    with pysam.VariantFile(vcffile.replace('vcf.gz','CRISPR-annotated.vcf.gz'),'w',header=vcfreader.header) as vcfwriter:
        
        for rec in vcfreader:
            
            chrom=rec.chrom
            chrom_nochr=chrom.replace('chr','').replace('M','MT')
            gene=rec.info['GENEINFO']
            pos=rec.pos
            ref=rec.ref
            alt=rec.alts[0]
            #af=rec.info['AF_EXAC']
            
            site=hg[chrom_nochr][int(pos)-w-1:int(pos)+w]
            r=hg[chrom_nochr][int(pos)-1]
            downstream=hg[chrom_nochr][int(pos)-w:int(pos)-1]
            upstream=hg[chrom_nochr][int(pos):int(pos)+w-1]
            
            rpclosest=100
            for pam in posPAM:
                p=downstream.find(pam)
                if p!=-1:
                    rp=w-pamsize-p
                    if rp<rpclosest:
                        strandorientation='+'
                        rpclosest=rp
                        pamstart=int(pos)-rpclosest-4
                        reftarget=hg[chrom_nochr][pamstart:pamstart+25]
                        alttarget=reftarget[:4+(rpclosest-1)]+alt+reftarget[4+rpclosest:]
                        
                    #print(n,chrom,gene,af,ref,alt,pos,"is targetable with CAS12 through PAM site: %s at position +%d!"%(pam,w-pamsize-p),downstream,r,upstream)
            
            for pam in negPAM:
                p=upstream.find(pam)
                if p!=-1:
                    rp=p+1
                    if rp<rpclosest:
                        strandorientation='-'
                        rpclosest=rp
                        pamstart=int(pos)-1+rpclosest+4
                        reftarget=revcomp(hg[chrom_nochr][pamstart-25:pamstart])
                        alttarget=reftarget[:4+(rpclosest-1)]+revcomp(alt)+reftarget[4+rpclosest:]
                    
                    #print(n,chrom,gene,af,ref,alt,pos,"is targetable with CAS12 through PAM site: %s at position +%d!"%(pam,p+1),downstream,r,upstream)
            
            rec.info['SNP_position']='PAM+%d'%rpclosest
            rec.info['CRISPR_ref_target_sequence']=reftarget.upper()
            rec.info['CRISPR_alt_target_sequence']=alttarget.upper()
            rec.info['STRAND']=strandorientation
            
            vcfwriter.write(rec)

There should now be a "clinvar_20210731.pathogenic.snps.CAS12targetable.CRISPR-annotated.vcf.gz" file which contains annotations (e.g. SNP_position (relative to PAM site), target sequences for both alleles of the SNP and the strand information) for all pathogenic SNPs in clinvar that are targetable by CAS12 